In [10]:
from datetime import date
from data_manager import DataManager
from scipy.stats import median_abs_deviation
import pandas as pd


In [11]:
self = DataManager()

def get_average_team_stat(self, df, last_n_team_games):
    teams = self.query_teams()
    
    # Initialize columns outside the loop
    stats = ['fgm', 'fga', 'fg3m', 'fg3a', 'points', 'rebounds', 'assists', 'steals', 'blocks', "to"]
    for stat in stats:
        df.loc[:, f'team_{stat}_mad'] = None
        df.loc[:, f'team_{stat}_average'] = None

    for team in df['TEAM'].unique():
        team_id = [team_object.id for team_object in teams if team_object.nickname == team][0]
        print(team_id)
        team_games = self.get_and_save_team_data(team_id).sort_values(by='date', ascending=False).head(last_n_team_games)
        print(team_games.columns)
        for stat in stats:
            total_points_average = team_games[stat].mean()
            mad_points = median_abs_deviation(team_games[stat], scale="normal")
            df.loc[df['TEAM'] == team, f'team_{stat}_average'] = total_points_average
            df.loc[df['TEAM'] == team, f'team_{stat}_mad'] = mad_points
            
    return df

In [12]:

analyzed_props = self.get_analyzed_props()
analyzed_prop_records = pd.DataFrame.from_dict([prop.entry for prop in analyzed_props])
profitable_props = analyzed_prop_records[analyzed_prop_records['EV'] > 0]


Received raw input.
Loading points props.
Loading assists props.
Loading fg3m props.
Loading rebounds props.
Loading fgm props.
Loading steals props.
Loading blocks props.
          player_name     team      stat  over_threshold  over_odds  \
0       Derrick White  Celtics    points            15.5       -120   
1        Jaylen Brown  Celtics    points            26.5       -110   
2    Payton Pritchard  Celtics    points             8.5       -115   
3          Al Horford  Celtics    points            10.5       -105   
4        Jrue Holiday  Celtics    points            13.5       -125   
5        Jayson Tatum  Celtics    points            28.5       -125   
6       Aaron Nesmith   Pacers    points            12.5       -105   
7          Obi Toppin   Pacers    points            10.5       -110   
8        Myles Turner   Pacers    points            17.5       -105   
9       Pascal Siakam   Pacers    points            23.5       -115   
10    Andrew Nembhard   Pacers    points       

In [13]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
view = profitable_props[['PLAYER', 'TEAM', 'STAT', 'TYPE']]
last_n_team_games = 25
df = get_average_team_stat(self, profitable_props, last_n_team_games=last_n_team_games)

2
Index(['team_name', 'points', 'rebounds', 'assists', 'efg', 'fg3a', 'fg3m',
       'fg3_pct', 'fga', 'fgm', 'fta', 'ft_pct', 'steals', 'blocks', 'to',
       'date', 'pace', 'def_rating', 'e_def_rating', 'off_rating',
       'e_off_rating'],
      dtype='object')
18
Index(['team_name', 'points', 'rebounds', 'assists', 'efg', 'fg3a', 'fg3m',
       'fg3_pct', 'fga', 'fgm', 'fta', 'ft_pct', 'steals', 'blocks', 'to',
       'date', 'pace', 'def_rating', 'e_def_rating', 'off_rating',
       'e_off_rating'],
      dtype='object')


C:\Users\rusta\AppData\Local\Temp\ipykernel_13984\4244349758.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'team_{stat}_mad'] = None
C:\Users\rusta\AppData\Local\Temp\ipykernel_13984\4244349758.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, f'team_{stat}_average'] = None
C:\Users\rusta\AppData\Local\Temp\ipykernel_13984\4244349758.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [14]:
print(df) 

df = df.sort_values(by="EV", ascending=False).reset_index(drop=True)
df.to_csv("temp.csv")
print(df.columns)


               PLAYER     TEAM      STAT  THRESH  ODDS   TYPE      PROB  \
0       Derrick White  Celtics    points    15.5  -120   over  0.639343   
3        Jaylen Brown  Celtics    points    26.5  -120  under  0.674605   
4    Payton Pritchard  Celtics    points     8.5  -115   over  0.855179   
9        Jrue Holiday  Celtics    points    13.5  -105  under  0.735040   
11       Jayson Tatum  Celtics    points    28.5  -105  under  0.696740   
13      Aaron Nesmith   Pacers    points    12.5  -125  under  0.688697   
14         Obi Toppin   Pacers    points    10.5  -110   over  0.582318   
19      Pascal Siakam   Pacers    points    23.5  -115  under  0.682127   
21    Andrew Nembhard   Pacers    points    13.5  -115  under  0.722967   
23     T.J. McConnell   Pacers    points    14.5  -115  under  0.730842   
25     Isaiah Jackson   Pacers    points     4.5  -130  under  0.628837   
27       Ben Sheppard   Pacers    points     5.5   100  under  0.518664   
29       Jayson Tatum  Ce

In [15]:
from collections import deque

teams = df['TEAM'].unique()
if len(teams) != 2:
    raise ValueError("The TEAM column must contain exactly two unique values for this operation.")

# Get the values to be swapped
value1 = df.loc[df['TEAM'] == teams[0], 'team_to_average'].values[0]
value2 = df.loc[df['TEAM'] == teams[1], 'team_to_average'].values[0]

# Create a mapping to swap the values
mapping = {teams[0]: value2, teams[1]: value1}

# Apply the mapping to the 'team_average_to' column
df['team_to_average'] = df['TEAM'].map(mapping)

# Rename the column
df.rename(columns={'team_to_average': 'oppteam_to_average'}, inplace=True)

    

In [16]:
team_stats_cols = [col for col in df.columns if "average" in col]
team_stats = df.copy()[team_stats_cols]
new_cols = [col.split("_")[1] for col in team_stats.columns]
team_stats.columns = new_cols
team_stats['TEAM'] = df['TEAM']


initial_availability = {}
for team in team_stats['TEAM'].unique():
    initial_availability[team] = dict(team_stats[team_stats['TEAM'] == team].reset_index(drop=True).iloc[0, :])



                

In [17]:

import copy

parlay_length = 8
parlay_count = 10
used = []
parlays = []
for i in range(parlay_count):
    availability = copy.deepcopy(initial_availability)
    parlay = []
    counter = 0
    for i, row in df.iterrows():
        if len(parlay) == parlay_length:
            break
        if i in used:
            continue
        else:
            if row['STAT'] in new_cols:
                stat = row['STAT']
                if row['TYPE'] == "over" and availability[row['TEAM']][stat] - (row['THRESH'] * 3) > 0:
                    availability[row['TEAM']][stat] -= row['THRESH']
                    parlay.append(i)
                    used.append(i)
                elif row['TYPE'] == "under" and availability[row['TEAM']][stat] + row['THRESH'] < initial_availability[row['TEAM']][stat]:
                    availability[row['TEAM']][stat] += row['THRESH']
                    parlay.append(i)
                    used.append(i)
    print(parlay)
    parlays.append(parlay)
parlays = [parlay for parlay in parlays if len(parlay) == parlay_length]
print(parlays)


                    

[0, 2, 3, 4, 5, 9, 16, 28]
[33, 46, 47, 50, 54, 58, 59, 60]
[64, 66, 71, 73, 74, 75]
[]
[]
[]
[]
[]
[]
[]
[[0, 2, 3, 4, 5, 9, 16, 28], [33, 46, 47, 50, 54, 58, 59, 60]]


In [18]:
parlay_dfs = []
# Segment the original DataFrame
for indexes in parlays:
    parlay_df = df.iloc[indexes]
    parlay_dfs.append(parlay_df)

# Display the resulting DataFrames
for i, segment_df in enumerate(parlay_dfs):
    print(f"Parlay {i}:\n{segment_df}\n")
    segment_df.to_csv(f"parlays/{date.today()}_{i}_greedy.csv")

Parlay 0:
              PLAYER     TEAM      STAT  THRESH  ODDS  TYPE      PROB  \
0   Payton Pritchard  Celtics    points     8.5  -115  over  0.855179   
2         Sam Hauser  Celtics      fg3m     1.5   130  over  0.655074   
3   Payton Pritchard  Celtics       fgm     3.5   115  over  0.699657   
4         Sam Hauser  Celtics       fgm     1.5  -120  over  0.818072   
5   Payton Pritchard  Celtics   assists     2.5  -120  over  0.814858   
9         Sam Hauser  Celtics  rebounds     2.5   135  over  0.603151   
16        Sam Hauser  Celtics    steals     0.5   170  over  0.472708   
28        Sam Hauser  Celtics   assists     0.5  -130  over  0.673720   

          EV  HOUSE_PROB team_fgm_mad team_fgm_average team_fga_mad  \
0   2.994064    0.534884     5.930409             42.8     7.413011   
2   2.533356    0.434783     5.930409             42.8     7.413011   
3   2.521312    0.465116     5.930409             42.8     7.413011   
4   2.498998    0.545455     5.930409           